# Bootstrap conf int test

In [0]:
from __future__ import division
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Let's clarify the rule of three sigma. Statement: 99.7% of the probabilistic mass of the random variable X∼N (μ, σ2) lies in the range μ ± c⋅σ. What is the exact value of the constant c equal to? Round the answer to four digits after the decimal point.

In [2]:
from scipy import stats
print('Answer: %.4f' % stats.norm.ppf(1-0.003/2))

Answer: 2.9677


In a five-year randomized study at Harvard Medical School, 11037 subjects took aspirin every other day, and another 11034 took a placebo. The study was blind, that is, the subjects did not know what they were taking.

For 5 years, a heart attack occurred in 104 subjects taking aspirin, and in 189 taking placebo.

Estimate how much the likelihood of a heart attack decreases with aspirin. Round the answer to four digits after the decimal point.

In [0]:
aspirin_data = np.array( [1 if i<104 else 0 for i in range(11037)] )
placebo_data = np.array( [1 if i<189 else 0 for i in range(11034)] )

In [4]:
prob_infarction_aspirin = aspirin_data.sum() / aspirin_data.shape[0]
prob_infarction_placebo = placebo_data.sum() / placebo_data.shape[0]
print('Infarction probability (aspirin): %.4f' % prob_infarction_aspirin)
print('Infarction probability (placebo): %.4f' % prob_infarction_placebo)
print('Infarction probability decrease: %.4f' % (prob_infarction_placebo - prob_infarction_aspirin))

Infarction probability (aspirin): 0.0094
Infarction probability (placebo): 0.0171
Infarction probability decrease: 0.0077


Now calculate a 95% confidence interval to reduce the likelihood of a heart attack when taking aspirin. What is it's upper bound equal to? Round the answer to four digits after the decimal point.

In [0]:
def proportions_confint_diff_ind(sample1, sample2, alpha=0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [6]:
print('Confidence interval: [%.4f, %.4f]' % proportions_confint_diff_ind(placebo_data, aspirin_data))

Confidence interval: [0.0047, 0.0107]


Let's continue to analyze the data from an experiment at Harvard Medical School.

For Bernoulli random variables X∼Ber (p), the value p1 − p, which is called odds, is often calculated. To estimate the chances of the sample, substitute p ^ with p ^. For example, the chances of a heart attack in the control group taking a placebo can be estimated as

Estimate how many times the chances of a heart attack are reduced with regular use of aspirin. Round the answer to four digits after the decimal point.

In [0]:
def odds(data):
    p = data.sum() / data.shape[0]
    return p / (1 - p)

In [8]:
odds_aspirin = odds(aspirin_data)
print('Odds aspirin: %.4f' % odds_aspirin)
odds_placebo = odds(placebo_data)
print('Odds aspirin: %.4f' % odds_placebo)

Odds aspirin: 0.0095
Odds aspirin: 0.0174


In [9]:
print('Times decrease of infarction: %.4f' % (odds_placebo / odds_aspirin))

Times decrease of infarction: 1.8321


The value that you calculated in the previous question is called the odds ratio. Build a 95% confidence interval for the odds ratio using the bootstrap. What is its lower boundary equal to? Round the answer to 4 digits after the decimal point.

To get exactly the same confidence interval as ours:

* compose outcome vectors in the control and test samples so that all units go first, and then all zeros;

* set random seed = 0;

make 1000 pseudo-samples from each patient group using the get_bootstrap_samples function.

In [0]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [0]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [0]:
np.random.seed(0)
odds_aspirin_data = np.array(list(map(odds, get_bootstrap_samples(aspirin_data, 1000))))
odds_placebo_data = np.array(list(map(odds, get_bootstrap_samples(placebo_data, 1000))))

In [19]:
print('95%% confidence interval for times decrease of infarction: %s' %
      str(stat_intervals(odds_placebo_data / odds_aspirin_data, 0.05)))

95% confidence interval for times decrease of infarction: [1.44419465 2.34321168]
